In [1]:
import pika
import numpy as np
import json
from sklearn.datasets import load_diabetes

In [2]:
X, y = load_diabetes(return_X_y=True)
random_row = np.random.randint(0, X.shape[0]-1)

In [3]:
random_row 

394

In [4]:
X[random_row]

array([ 0.03444337, -0.04464164,  0.01858372,  0.05630106,  0.01219057,
       -0.05454912, -0.06917231,  0.07120998,  0.13008061,  0.00720652])

In [5]:
y[random_row]

273.0

In [6]:
# Подключение к серверу на локальном хосте:
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

In [7]:
# Создадим очередь, с которой будем работать:
channel.queue_declare(queue='y_true')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=1', 'message_count=0', 'queue=y_true'])>"])>

In [8]:
# Опубликуем сообщение
# exchange определяет, в какую очередь отправляется сообщение. Если мы используем дефолтную точку обмена, то значение можно оставить пустым.
# параметр routing_key указывает имя очереди, 
# параметр body тело самого сообщения, 
channel.basic_publish(exchange='',
                      routing_key='y_true',
                      body=json.dumps(y[random_row]))


print('Сообщение с правильным ответом отправлено в очередь')

Сообщение с правильным ответом отправлено в очередь


In [9]:
channel.queue_declare(queue='Features')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=1', 'message_count=0', 'queue=Features'])>"])>

In [10]:
channel.basic_publish(exchange='',
                      routing_key='Features',
                      body=json.dumps(list(X[random_row])))

print('Сообщение с признаками отправлено в очередь')

Сообщение с признаками отправлено в очередь


In [11]:
# Закроем подключение 
connection.close()

In [12]:
json.dumps(y[random_row])

'273.0'

In [13]:
json.dumps(list(X[random_row]))

'[0.0344433679824045, -0.044641636506989, 0.0185837235634525, 0.0563010619323185, 0.0121905687618, -0.0545491159304391, -0.0691723102806364, 0.0712099797536354, 0.130080609521753, 0.00720651632920303]'